# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Defaulting to user installation because normal site-packages is not writeable


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [10]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [11]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

In [12]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
]

In [13]:
llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    api_key="sk-fYThF6OhEdD6QgFyCeB0C820728147439283Bc5529BeFd68",
    base_url="https://api.xiaoai.plus/v1"
)

In [14]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [15]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [16]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [17]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [18]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题']


In [19]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题


In [20]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [21]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题

<< INPUT >>
{input}

<< OUTPUT (must include ```json at the start of the

In [22]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [23]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指一个理想的热辐射体，它能够完全吸收所有辐射能量并以各种波长发射出来。根据普朗克定律，黑体辐射的能量密度与波长有关，其能量密度随着波长的增加而下降，而其最大值则出现在特定的波长上，这就是所谓的黑体辐射谱。黑体辐射在物理学和天文学中都有重要的应用，它们可以用来解释星体的辐射特性以及宇宙的背景辐射。'}


In [24]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要确定什么是质数。质数是只能被1和自身整除的数，也就是除了1和它本身，没有其他因数的数。\n\n接下来，我们要找出40的所有因数。40可以被1、2、4、5、8、10、20和40整除。\n\n然后，我们需要找出这些因数中哪些是质数。1不是质数，因为它只有一个因数，所以我们可以排除它。2、5、8和10都不是质数，因为它们都可以被其他数字整除。但是4、20和40都是质数，因为它们除了1和自身，没有其他因数。\n\n现在，我们要找出其中能被3整除的质数。4加1是5，20加1是21，40加1是41，所以这三个质数都不能被3整除。\n\n最后，我们可以得出结论：40的'}


In [25]:
router_chain.verbose = True

In [26]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是一种极端的天体，它有着非常强大的引力，以至于连光都无法逃脱它的吸引。它的存在是由于质量非常大的物体（例如恒星）在死亡时，会压缩到极小的体积，从而形成一个巨大的引力场。这个引力场可以吸收周围的物质，甚至连光也无法逃脱。因此，黑洞也被称为“吞噬一切的天体”。目前，科学家们还在探索黑洞的奥秘，它们也被认为是宇宙中最神秘的现象之一。希望这个简洁的回答能够解答你的疑问。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [28]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位经验丰富的生物学家。
你对生物学领域有深入的了解，能够回答从分子生物学到生态学的各种问题。
当面对未知的生物学现象时，你会基于现有的科学知识进行合理的推测。

这是一个问题：
{input}"""

computer_science_template = """你是一位资深的计算机科学家。
你精通编程、算法和系统设计。你能够清晰地解释复杂的技术概念，并解决技术问题。
如果遇到超出你专业领域的问题，你会寻求最新的研究或专家意见。

这是一个问题：
{input}"""

chinese_literature_template = """你是一位汉语言文学的专家。
你对中国古典文学和现代文学都有深刻的理解，能够解读文学作品中的深层含义。
面对文学作品的解析，你会引用相应的文献和理论来支持你的观点。

这是一个问题：
{input}"""

prompt_infos = [
    # 原有的物理和数学提示信息
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    # 新增的生物老师提示信息
    {
        "name": "生物",
        "description": "适用于回答生物学问题",
        "prompt_template": biology_template,
    },
    # 新增的计算机老师提示信息
    {
        "name": "计算机",
        "description": "适用于回答计算机科学和编程问题",
        "prompt_template": computer_science_template,
    },
    # 新增的汉语言文学老师提示信息
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学相关问题",
        "prompt_template": chinese_literature_template,
    },
]

# 注意：在实际的代码实现中，`physics_template`、`math_template`、`biology_template`、
# `computer_science_template` 和 `chinese_literature_template` 需要被定义为相应的模板字符串或函数。


llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    api_key="sk-fYThF6OhEdD6QgFyCeB0C820728147439283Bc5529BeFd68",
    base_url="https://api.xiaoai.plus/v1"
)

# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")


# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

print(chain.invoke("数据结构的几大类型"))



> Entering new MultiPromptChain chain...
计算机: {'input': '数据结构的几大类型'}
> Finished chain.
{'input': '数据结构的几大类型', 'text': '有哪些？\n\n答案：\n数据结构的几大类型包括：线性结构、树结构、图结构和哈希表结构。线性结构包括数组、链表、栈和队列；树结构包括二叉树、二叉搜索树、平衡二叉树等；图结构包括有向图和无向图；哈希表结构包括哈希表和哈希集合。每种数据结构都有自己的特点和适用场景，可以根据具体的需求来选择合适的数据结构。'}


In [29]:
print(chain.invoke("细胞壁有几层"))



> Entering new MultiPromptChain chain...
生物: {'input': '细胞壁有几层'}
> Finished chain.
{'input': '细胞壁有几层', 'text': '？\n\n细胞壁一般由两层构成，外层是细胞壁，内层是细胞膜。但是具体的细胞壁结构会因细胞类型和生物种类而有所不同。有些细胞壁可能会有多层，如植物细胞壁由原生质壁、次生质壁和细胞外基质组成，共有三层结构。'}


In [30]:
print(chain.invoke("笔画最多的汉字"))



> Entering new MultiPromptChain chain...
汉语言文学: {'input': '笔画最多的汉字'}
> Finished chain.
{'input': '笔画最多的汉字', 'text': '是哪个？\n\n答：根据《新华字典》记载，笔画最多的汉字是「龘」，共有57笔。它是一个古汉语中的字，意为「负」，现在已经很少使用。'}
